# Loading Dataset

In [53]:
import pandas as pd

# Load datasets
df1 = pd.read_csv("data_1.csv")
df2 = pd.read_csv("data_2.csv")
df3 = pd.read_csv("data_3.csv")

In [54]:
# Merge them by stacking (row-wise)
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

In [55]:
# Drop duplicates based on the "Title" column to keep only unique titles
df_unique = merged_df.drop_duplicates(subset="Title")

In [56]:
df = df_unique.dropna(subset=["Title"])

In [57]:
df

,Query Id,Query Name,Date,Title,Snippet,Url,Domain,Sentiment,Page Type,Language,...,Subreddit,Subreddit NSFW,Subreddit Subscribers,Subreddit Topics,Subscriptions,Tiktok Comments,Tiktok Likes,Tiktok Shares,Weblog Title,Emotion
0,2002928223,wildfire,2025-01-11 04:59:42.0,RT @npdcoalition Are you ready to manage burns? This resource provides burn management guidance for both children and adults. https://t.co/DshOBzGG9h #burns #wildfire @ACEPNow #MedTwittter @ENAorg @Ameriburn @hospitals4kids @ahahospitals @ENAorg @EMS1 https://t.co/duaX650J2F,RT @npdcoalition Are you ready to manage burns? This resource provides burn management guidance for both children and adults. https://t.co/DshOBzGG9h #burns #wildfire @ACEPNow #MedTwittter @ENAorg @Ameriburn @hospitals4kids @ahahospitals @ENAorg @EMS1 https://t.co/duaX650J2F,http://twitter.com/pfrostamis/statuses/1877943445396021270,twitter.com,neutral,twitter,en,...,NaN,False,NaN,NaN,NaN,0,0,0,"pfrostamis (Patricia Frost PNP, Pediatric Disasterologist)",NaN
1,2002928223,wildfire,2025-01-11 04:59:34.0,RT @MattWallace888 🚨🚨🚨 CALIFORNIA FIRE PLANE CAUGHT ON CAMERA SKIPPING OVER MIDDLE-CLASS NEIGHBORHOODS ⚠️\n\nOFFICIALS HAVE BEEN PRIORITIZING THE HOUSES OF THE ELITES WHILE THE MOST VULNERABLE DIE‼️\n\nRUMORS ARE CIRCULATING THEY ARE PLANNING TO USE THE BURNED AREAS TO BUILD APARTMENT COMPLEXES FOR…,RT @MattWallace888 🚨🚨🚨 CALIFORNIA FIRE PLANE CAUGHT ON CAMERA SKIPPING OVER MIDDLE-CLASS NEIGHBORHOODS ⚠️ OFFICIALS HAVE BEEN PRIORITIZING THE HOUSES OF THE ELITES WHILE THE MOST VULNERABLE DIE‼️ RUMORS ARE CIRCULATING THEY ARE PLANNING TO USE THE BURNED AREAS TO BUILD APARTMENT COMPLEXES FOR…,http://twitter.com/mbmenlo/statuses/1877943412013326445,twitter.com,neutral,twitter,en,...,NaN,False,NaN,NaN,NaN,0,0,0,mbmenlo (Mary Beth🇺🇸 🇺🇸🇺🇸),NaN
2,2002928223,wildfire,2025-01-11 04:59:32.0,"RT @dog_rates This dog was found near a devastating L.A. wildfire, covered in burns and in need of urgent care. @PasadenaHumane is treating him and over 300 other animals affected. You can help them provide critical resources for displaced animals by donating below ❤️\n\nhttps://t.co/15xqO4LLRX https://t.co/Rj2HojYFi0","RT @dog_rates This dog was found near a devastating L.A. wildfire, covered in burns and in need of urgent care. @PasadenaHumane is treating him and over 300 other animals affected. You can help them provide critical resources for displaced animals by donating below ❤️ https://t.co/15xqO4LLRX https://t.co/Rj2HojYFi0",http://twitter.com/Ogre_42/statuses/1877943404018962478,twitter.com,neutral,twitter,en,...,NaN,False,NaN,NaN,NaN,0,0,0,Ogre_42 (EMJ-Ogre_42),NaN
3,2002928223,wildfire,2025-01-11 04:59:17.0,RT @GavinNewsom California is preventing insurance companies from canceling home coverage for LA wildfire victims in impacted zip codes over the next year.\n\nThe folks in these communities have suffered enough. They should not have to deal with the stress of home insurance on top of it all.,RT @GavinNewsom California is preventing insurance companies from canceling home coverage for LA wildfire victims in impacted zip codes over the next year. The folks in these communities have suffered enough. They should not have to deal with the stress of home insurance on top of it all.,http://twitter.com/GellerLaurie/statuses/1877943339565117788,twitter.com,negative,twitter,en,...,NaN,False,NaN,NaN,NaN,0,0,0,GellerLaurie (Laurie Geller ☮️),NaN
4,2002928223,wildfire,2025-01-11 04:59:16.0,"RT @Weather_West Due to increased activity on eastern flank of #PalisadesFire, significant new evacuations have been issued. Mandatory orders now extend all way east to I-405 corridor (!), & there are even limited warnings across 405 into Bel Air & north to Ventura Blvd (!). [1/2] #CAwx #CAfire https://t.co/kUzfRzqG3Y","RT @Weather_West Due to increased activity on eastern flank of #PalisadesFire, significant new evacuations have been issued. Mandatory orders now extend all way east to I-405 corridor (!), & there are even limited wa

# Cleaning Dataset

## First Self Exploration

In [19]:
import re
import unicodedata
import pandas as pd
from typing import List

# ====== Load & Prepare Source ======
# Prefer the deduped, non-NA "Title" dataframe if it exists
df_source = df.copy()

# Use Title explicitly
df_source["raw_text"] = df_source["Title"].astype(str)

# ====== Regex Helpers ======
url_re = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
mention_re = re.compile(r"@\w+")
rt_re = re.compile(r"\bRT\b:?")          # remove "RT" (optional trailing colon)
hashtag_token_re = re.compile(r"#\w+", flags=re.UNICODE)  # remove hashtags from text to avoid tag influence
zw_nbsp_re = re.compile(r"[\u200B-\u200D\uFEFF\u00A0]")  # zero-width / NBSP

# ====== Cleaning Function (KEEP emojis) ======
def clean_text_title_llm(text: str) -> str:
    # 1) Unicode normalize
    text = unicodedata.normalize("NFKC", text)
    # 2) Remove zero-width & NBSP
    text = zw_nbsp_re.sub(" ", text)
    # 3) Normalize URLs and mentions
    text = url_re.sub(" [URL] ", text)
    text = mention_re.sub(" [USER] ", text)
    # 4) Remove retweet markers
    text = rt_re.sub(" ", text)
    # 5) Remove hashtags entirely (while we keep them separately)
    text = hashtag_token_re.sub(" ", text)
    # 6) Keep emojis as-is (no emoji stripping here)
    # 7) Whitespace tidy
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s*\n\s*", "\n", text)
    return text.strip()

# ====== Hashtag Extraction (saved separately to keep optional use) ======
# Simple extractor
hashtags_simple = df_source["raw_text"].str.findall(r"#([A-Za-z0-9_]+)")
# Unicode-friendly fallback: capture letters (excluding digits/underscore) to catch non-Latin words
hashtags_extra = df_source["raw_text"].str.findall(r"#([^\W_]+)", flags=re.UNICODE)

def merge_hashtags(a: List[str] | None, b: List[str] | None) -> list[str]:
    s = set((a or []) + (b or []))
    return sorted(s)

df_source["Hashtags"] = [merge_hashtags(a, b) for a, b in zip(hashtags_simple, hashtags_extra)]

# ====== Apply Cleaning ======
df_source["clean_text"] = df_source["raw_text"].apply(clean_text_title_llm)

output_path = "data_clean_title_llm.csv"
df_source.to_csv(output_path, index=False)

In [22]:
df_source

,Query Id,Query Name,Date,Title,Snippet,Url,Domain,Sentiment,Page Type,Language,...,Subreddit Subscribers,Subreddit Topics,Subscriptions,Tiktok Comments,Tiktok Likes,Tiktok Shares,Weblog Title,Emotion,raw_text,clean_text
0,2002928223,wildfire,2025-01-11 04:59:42.0,RT @npdcoalition Are you ready to manage burns...,RT @npdcoalition Are you ready to manage burns...,http://twitter.com/pfrostamis/statuses/1877943...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,"pfrostamis (Patricia Frost PNP, Pediatric Disa...",NaN,RT @npdcoalition Are you ready to manage burns...,[USER] Are you ready to manage burns? This res...
1,2002928223,wildfire,2025-01-11 04:59:34.0,RT @MattWallace888 🚨🚨🚨 CALIFORNIA FIRE PLANE C...,RT @MattWallace888 🚨🚨🚨 CALIFORNIA FIRE PLANE C...,http://twitter.com/mbmenlo/statuses/1877943412...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,mbmenlo (Mary Beth🇺🇸 🇺🇸🇺🇸),NaN,RT @MattWallace888 🚨🚨🚨 CALIFORNIA FIRE PLANE C...,[USER] 🚨🚨🚨 CALIFORNIA FIRE PLANE CAUGHT ON CAM...
2,2002928223,wildfire,2025-01-11 04:59:32.0,RT @dog_rates This dog was found near a devast...,RT @dog_rates This dog was found near a devast...,http://twitter.com/Ogre_42/statuses/1877943404...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,Ogre_42 (EMJ-Ogre_42),NaN,RT @dog_rates This dog was found near a devast...,[USER] This dog was found near a devastating L...
3,2002928223,wildfire,2025-01-11 04:59:17.0,RT @GavinNewsom California is preventing insur...,RT @GavinNewsom California is preventing insur...,http://twitter.com/GellerLaurie/statuses/18779...,twitter.com,negative,twitter,en,...,NaN,NaN,NaN,0,0,0,GellerLaurie (Laurie Geller ☮️),NaN,RT @GavinNewsom California is preventing insur...,[USER] California is preventing insurance comp...
4,2002928223,wildfire,2025-01-11 04:59:16.0,RT @Weather_West Due to increased activity on ...,RT @Weather_West Due to increased activity on ...,http://twitter.com/FreeSpkr/statuses/187794333...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,FreeSpkr (Cheri Hierbaum),NaN,RT @Weather_West Due to increased activity on ...,[USER] Due to increased activity on eastern fl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131263,2002928223,wildfire,2025-01-18 20:00:55.0,No Doubt to reunite for performance at FireAid...,No Doubt to reunite for performance at FireAid...,http://twitter.com/THR/statuses/18807069578220...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,THR (The Hollywood Reporter),NaN,No Doubt to reunite for performance at FireAid...,No Doubt to reunite for performance at FireAid...
131267,2002928223,wildfire,2025-01-18 20:00:03.0,@TMZ Full story 🔗 https://t.co/eIs7xovUh8,@TMZ Full story 🔗 https://t.co/eIs7xovUh8,http://twitter.com/TMZ/statuses/18807067394497...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,TMZ (TMZ),Joy,@TMZ Full story 🔗 https://t.co/eIs7xovUh8,[USER] Full story 🔗 [URL]
131268,2002928223,wildfire,2025-01-18 20:00:02.0,Vanessa Bryant gifted L.A. wildfire victims Ni...,Vanessa Bryant gifted L.A. wildfire victims Ni...,http://twitter.com/TMZ/statuses/18807067378476...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,TMZ (TMZ),NaN,Vanessa Bryant gifted L.A. wildfire victims Ni...,Vanessa Bryant gifted L.A. wildfire victims Ni...
131269,2002928223,wildfire,2025-01-18 20:00:01.0,JP Saxe Announces His Single Release Show Is N...,JP Saxe Announces His Single Release Show Is N...,http://twitter.com/StarlightPR1/statuses/18807...,twitter.com,neutral,twitter,en,...,NaN,NaN,NaN,0,0,0,StarlightPR1 (Starlight PR™),NaN,JP Saxe Announces His Single Release Show Is N...,JP Saxe Announces His Single Release Show Is N...


## Zihui Code

In [37]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 15.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
import re
import nltk
from nltk.corpus import stopwords

# download stopwords once
nltk.download('stopwords')
stopword = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afrai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
def clean_tweet(text):
    if isinstance(text, float):
        return ""

    text = text.lower()  #lowercase all letters
    text = text.replace("&#39;", "'")
    text = text.replace("&quot;", "")
    text = re.sub(r'http\S+', '', text)  #remove http links
    text = re.sub(r'www.\S+', '', text)  #remove www links
    text = re.sub(r'^rt\s+@\w+', '', text)
   # text = re.sub('(rt\s@[A-Za-z0-9_]+[A-Za-z0-9_]:+)', '', text) #remove retweet
    text = re.sub('(@[A-Za-z0-9_]+[A-Za-z0-9_]+)', '', text) #remove tweeted at
    text = re.sub(r'\w*\d\w*', '', text)  #remove numbers
    text = re.sub('_', '', text)  #remove '_'
    text = re.sub('[!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~]', ' ', text)  #remove punctuations
    #text = "".join([text.lower()for word in text if word not in string.punctuation]) # alter way to remove puntuation
    text = re.sub(r'\[.*?\]', '', text)  #remove text in square brackets
    text = re.sub("’", "", text)  # to avoid removing contractions in english
    text = re.sub("[^a-z0-9]", " ", text)  #remove all not number or letters
    text = re.split('\W+', text)  #token the text
    text = [word for word in text if not word in stopword]  #remove stop word
    #text = [ps.stem(word) for word in text if word not in stopword]
    text = " ".join(word for word in text)
    return text

<>:20: SyntaxWarning: invalid escape sequence '\W'
<>:20: SyntaxWarning: invalid escape sequence '\W'
C:\Users\afrai\AppData\Local\Temp\ipykernel_3988\723154681.py:20: SyntaxWarning: invalid escape sequence '\W'
  text = re.split('\W+', text)  #token the text


In [60]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [61]:
import numpy as np

In [62]:
df["cleaned_tweets"] = df["Title"].apply(lambda x: remove_emojis(clean_tweet(x)))

C:\Users\afrai\AppData\Local\Temp\ipykernel_3988\2728348674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleaned_tweets"] = df["Title"].apply(lambda x: remove_emojis(clean_tweet(x)))


In [63]:
pd.set_option("display.max_colwidth", None)   # show full text in DataFrame columns

df["cleaned_tweets"].head()

0                                                                                                ready manage burns resource provides burn management guidance children adults burns wildfire medtwittter 
1      california fire plane caught camera skipping middle class neighborhoods officials prioritizing houses elites vulnerable die rumors circulating planning use burned areas build apartment complexes 
2                                               dog found near devastating l wildfire covered burns need urgent care treating animals affected help provide critical resources displaced animals donating 
3                     california preventing insurance companies canceling home coverage la wildfire victims impacted zip codes next year folks communities suffered enough deal stress home insurance top 
4     due increased activity eastern flank palisadesfire significant new evacuations issued mandatory orders extend way east corridor even limited warnings across bel air north ventura blv

In [64]:
df.to_csv("cleaned_data.csv", index = False)

## Merge with predicted data

In [2]:
import pandas as pd

In [3]:
df_predict = pd.read_csv("final_merged.csv")

In [4]:
cleaned_data = pd.read_csv("cleaned_data.csv")

In [5]:
final_cols_to_add = [
    "Title",
    "Predicted Location",
    "Predicted Sentiment",
    "Predicted related",
    "Predicted location city",
    "Count"
]

# Subset final_merged
final_subset = df_predict[final_cols_to_add]

# Merge cleaned_data with only selected columns from final_merged
merged_slim = pd.merge(cleaned_data, final_subset, on="Title", how="inner")

In [7]:
merged_slim.to_csv("cleaned_data_join_predict.csv", index = False)